In [1]:
import torch
import torch.nn as nn

In [2]:
a = torch.IntTensor([1, 1, 2, 3, 4])

In [3]:
b = torch.IntTensor([1, 1, 4, 5, 6])
a * b

tensor([ 1,  1,  8, 15, 24], dtype=torch.int32)

In [4]:
torch.dot(a, b)

tensor(49, dtype=torch.int32)

In [5]:
a = torch.rand((1, 4))
b = torch.rand((4, 1))
a*b

tensor([[0.1934, 0.5731, 0.0477, 0.2508],
        [0.0904, 0.2679, 0.0223, 0.1172],
        [0.0871, 0.2581, 0.0215, 0.1130],
        [0.1268, 0.3757, 0.0313, 0.1644]])

In [6]:
a.shape

torch.Size([1, 4])

In [7]:
a[None, :].shape

torch.Size([1, 1, 4])

In [8]:
a.view(-1, 1).shape

torch.Size([4, 1])

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv("iris.data", names=['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width', 'Class'])
df.head()

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [11]:
torch.Tensor(df.loc[1][:-1].to_numpy().astype("float32"))

tensor([4.9000, 3.0000, 1.4000, 0.2000])

In [12]:
#one_hoter = {'Iris-setosa': [[1], [0], [0]], "Iris-virginica": [[0], [0], [1]], "Iris-versicolor": [[0], [1], [0]]}
one_hoter = {'Iris-setosa': [1, 0, 0], "Iris-virginica": [0, 0, 1], "Iris-versicolor": [0, 1, 0]}

In [13]:
from torch.utils.data import Dataset, DataLoader

In [14]:
class IrisDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame):
        self.df = dataframe
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, ind):
        raw = self.df.loc[ind].to_numpy()
        target = raw[-1]
        target = torch.Tensor(one_hoter[target])
        samples = torch.Tensor(raw[:-1].astype("float32"))
        return samples, target

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(4, 32)
        self.f_act = nn.ReLU()
        self.linear2 = nn.Linear(32, 3)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, X):
        #norm = self.norm(X)
        res = self.linear1(X)
        res = self.f_act(res)
        res = self.linear2(res)
        res = self.softmax(res)
        return res

In [21]:
from torchsummary import summary
summary(Net())

Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            160
├─ReLU: 1-2                              --
├─Linear: 1-3                            99
├─Softmax: 1-4                           --
Total params: 259
Trainable params: 259
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            160
├─ReLU: 1-2                              --
├─Linear: 1-3                            99
├─Softmax: 1-4                           --
Total params: 259
Trainable params: 259
Non-trainable params: 0

In [17]:
def train_one_epoch(epoch, trainloader, optimizer, model, criterion, dataset):
    correct = 0
    epoch_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        x, y = data
        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        epoch_loss += outputs.shape[0] * loss.item()
        optimizer.step()
        outputs = (outputs > 0.5).float()
        correct += (outputs == y).float().sum()
    print(
        f'TRAIN [{epoch + 1}] loss: {epoch_loss / dataset.__len__():.3f}')

In [18]:
dataset = IrisDataset(df)
trainloader = DataLoader(dataset, batch_size=32,  shuffle=True)
net = Net()
optimizer = torch.optim.RMSprop(params=net.parameters())
loss = nn.CrossEntropyLoss()

In [19]:
for epoch in range(20):
    train_one_epoch(epoch, trainloader, optimizer, net, loss, dataset)

TRAIN [1] loss: 1.044
TRAIN [2] loss: 0.893
TRAIN [3] loss: 0.808
TRAIN [4] loss: 0.762
TRAIN [5] loss: 0.752
TRAIN [6] loss: 0.729
TRAIN [7] loss: 0.718
TRAIN [8] loss: 0.700
TRAIN [9] loss: 0.673
TRAIN [10] loss: 0.661
TRAIN [11] loss: 0.702
TRAIN [12] loss: 0.644
TRAIN [13] loss: 0.650
TRAIN [14] loss: 0.674
TRAIN [15] loss: 0.615
TRAIN [16] loss: 0.641
TRAIN [17] loss: 0.624
TRAIN [18] loss: 0.619
TRAIN [19] loss: 0.617
TRAIN [20] loss: 0.614


In [20]:
net(torch.Tensor(df.loc[1][:-1].to_numpy().astype("float32"))[None, :])

tensor([[9.9290e-01, 7.0958e-03, 1.4098e-09]], grad_fn=<SoftmaxBackward0>)

In [24]:
net.eval()

Net(
  (linear1): Linear(in_features=4, out_features=32, bias=True)
  (f_act): ReLU()
  (linear2): Linear(in_features=32, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)

In [ ]:
net(data)